In [1]:
import numpy as np
import pandas as pd

from utils import read_pickles, rmse
from utils_score import WRMSSEEvaluator, calc_score_cv

pd.set_option('display.max_columns', 100)

%matplotlib inline

In [6]:
# load csv files
df = pd.read_csv('../input/sales_train_evaluation.csv')
calendar = pd.read_csv('../input/calendar.csv')
prices = pd.read_csv('../input/sell_prices.csv')
sample_sub = pd.read_csv("../input/sample_submission.csv")

sample_sub["order"] = range(sample_sub.shape[0])

In [2]:
oof = pd.read_csv('../output/oof_lgbm_cv_28days.csv')

oof_foods = pd.read_csv('../output/oof_lgbm_cv_foods.csv')
oof_household = pd.read_csv('../output/oof_lgbm_cv_household.csv')
oof_hobbies = pd.read_csv('../output/oof_lgbm_cv_hobbies.csv')

oof_foods_g = pd.read_csv('../output/oof_lgbm_group_k_fold_foods.csv')
oof_household_g = pd.read_csv('../output/oof_lgbm_group_k_fold_household.csv')
oof_hobbies_g = pd.read_csv('../output/oof_lgbm_group_k_fold_hobbies.csv')

In [3]:
oof = pd.read_csv('../output/oof_lgbm_cv_28days.csv')
scores = calc_score_cv(oof)

 92%|█████████▏| 11/12 [00:05<00:00,  1.31it/s]/media/fujiwara/2THDD/M5-Forecasting-Accuracy/src/utils_score.py:56: RuntimeWarning: Mean of empty slice.
  scale.append(((series[1:] - series[:-1]) ** 2).mean())
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 12/12 [00:10<00:00,  1.16it/s]


In [4]:
end_train=1941
cols_valid1 = [f'd_{c}' for c in range(end_train-28+1,end_train+1)]
cols_valid2 = [f'd_{c}' for c in range(end_train-28*2+1,end_train-28+1)]
cols_valid2 = [f'd_{c}' for c in range(end_train-365+1,end_train-365+28+1)]

In [6]:
round(np.mean(scores),6)

0.618596

In [7]:
df_train1 = df.iloc[:, :-28]
df_valid1 = df.iloc[:, -28:]

evaluator1 = WRMSSEEvaluator(df_train1, df_valid1, calendar, prices)

df_train2 = df.iloc[:, :-28*2]
df_valid2 = df.iloc[:, -28*2:-28]

evaluator2 = WRMSSEEvaluator(df_train2, df_valid2, calendar, prices)

df_train3 = df.iloc[:, :-365]
df_valid3 = df.iloc[:, -365:-365+28]

evaluator3 = WRMSSEEvaluator(df_train3, df_valid3, calendar, prices)

 92%|█████████▏| 11/12 [00:05<00:00,  1.33it/s]/media/fujiwara/2THDD/M5-Forecasting-Accuracy/src/utils_score.py:56: RuntimeWarning: Mean of empty slice.
  scale.append(((series[1:] - series[:-1]) ** 2).mean())
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 12/12 [00:10<00:00,  1.20it/s]


In [5]:
oof = pd.melt(oof,id_vars='id',var_name='d',value_name='demand')

In [6]:
oof['d_numeric']=oof['d'].apply(lambda x: str(x)[2:]).astype(int)
valid1 = oof[(oof['d_numeric']>1941-28)&(oof['d_numeric']<=1941)]
valid2 = oof[(oof['d_numeric']>1941-28*2)&(oof['d_numeric']<=1941-28)]
valid3 = oof[(oof['d_numeric']>1941-365)&(oof['d_numeric']<=1941-365+28)]

In [7]:
valid1 = valid1[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()
valid2 = valid2[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()
valid3 = valid3[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()

In [8]:
cols_d1 = [c for c in valid1.columns if 'd_' in c]
valid1 = valid1.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator1.score(valid1[cols_d1])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.5078660464328384
cat_id 0.5128658824289202
state_id 0.5185990485844362
dept_id 0.546743222367672
store_id 0.5865948380721752
item_id 0.8575593943805533
['state_id', 'cat_id'] 0.5440950528493967
['state_id', 'dept_id'] 0.578590245245477
['store_id', 'cat_id'] 0.6185037667846648
['store_id', 'dept_id'] 0.6645376682960282
['item_id', 'state_id'] 0.8528208753122966
['item_id', 'store_id'] 0.8464674726840481
0.6362702927865422


In [9]:
cols_d2 = [c for c in valid2.columns if 'd_' in c]
valid2 = valid2.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator2.score(valid2[cols_d2])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.23876481681170472
cat_id 0.2932664800507613
state_id 0.3309030441526736
dept_id 0.3757801475414047
store_id 0.4552217259552134
item_id 0.8431094842694788
['state_id', 'cat_id'] 0.39848028144125025
['state_id', 'dept_id'] 0.47419660613327347
['store_id', 'cat_id'] 0.5323033338349706
['store_id', 'dept_id'] 0.5938330362618495
['item_id', 'state_id'] 0.8511709197879268
['item_id', 'store_id'] 0.8464000135889933
0.519452490819125


In [10]:
cols_d3 = [c for c in valid3.columns if 'd_' in c]
valid3 = valid3.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator3.score(valid3[cols_d3])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.5351364652296812
cat_id 0.5585523063098284
state_id 0.6247569109831275
dept_id 0.5808966657090066
store_id 0.6751412563742194
item_id 0.9373676687454482
['state_id', 'cat_id'] 0.6381713151941524
['state_id', 'dept_id'] 0.6700222435604216
['store_id', 'cat_id'] 0.698149278512715
['store_id', 'dept_id'] 0.7348639271569829
['item_id', 'state_id'] 0.8953994576864794
['item_id', 'store_id'] 0.8523123679951605
0.7000641552881018


In [11]:
np.mean([0.6362702927865422,0.519452490819125,0.7000641552881018])

0.6185956462979231

In [15]:
oof2 = oof_foods.append(oof_household)
oof2 = oof2.append(oof_hobbies)

In [29]:
oof2['d_numeric']=oof2['d'].apply(lambda x: str(x)[2:]).astype(int)
valid2_1 = oof2[(oof2['d_numeric']>1941-28)&(oof2['d_numeric']<=1941)]
valid2_2 = oof2[(oof2['d_numeric']>1941-28*2)&(oof2['d_numeric']<=1941-28)]
valid2_3 = oof2[(oof2['d_numeric']>1941-365)&(oof2['d_numeric']<=1941-365+28)]

In [30]:
valid2_1 = valid2_1[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()
valid2_2 = valid2_2[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()
valid2_3 = valid2_3[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()

In [37]:
valid2 = oof2[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()

In [31]:
valid2_1 = valid2_1.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator1.score(valid2_1[cols_d1])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.5121444185250336
cat_id 0.5235582464825189
state_id 0.5272711967653047
dept_id 0.5534818341478575
store_id 0.5942724591602403
item_id 0.8689089732407096
['state_id', 'cat_id'] 0.5630643276436076
['state_id', 'dept_id'] 0.5983907118039524
['store_id', 'cat_id'] 0.6378993022226473
['store_id', 'dept_id'] 0.6830731891455724
['item_id', 'state_id'] 0.8597160607222496
['item_id', 'store_id'] 0.8500155098176712
0.6476496858064471


In [38]:
valid2 = valid2.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator1.score(valid2[cols_d1])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.5121444185250336
cat_id 0.5235582464825189
state_id 0.5272711967653047
dept_id 0.5534818341478575
store_id 0.5942724591602403
item_id 0.8689089732407096
['state_id', 'cat_id'] 0.5630643276436076
['state_id', 'dept_id'] 0.5983907118039524
['store_id', 'cat_id'] 0.6378993022226473
['store_id', 'dept_id'] 0.6830731891455724
['item_id', 'state_id'] 0.8597160607222496
['item_id', 'store_id'] 0.8500155098176712
0.6476496858064471


In [32]:
valid2_2 = valid2_2.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator2.score(valid2_2[cols_d2])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.2517976415467098
cat_id 0.33776027901238787
state_id 0.34024402784247093
dept_id 0.41735888781365443
store_id 0.4393869525687345
item_id 0.8436682976169463
['state_id', 'cat_id'] 0.42958707634931004
['state_id', 'dept_id'] 0.507937845501746
['store_id', 'cat_id'] 0.5299036093834514
['store_id', 'dept_id'] 0.5977077457508813
['item_id', 'state_id'] 0.8513368254570093
['item_id', 'store_id'] 0.8465023907433411
0.5327659649655535


In [ ]:
valid2_2 = valid2_2.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator2.score(valid2_2[cols_d2])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

In [39]:
groups, scores = evaluator3.score(valid2[cols_d3])
for g, s in zip(groups, scores):
    print(g,s)

print(np.mean(scores))

all_id 0.5447619502922475
cat_id 0.5529630486180929
state_id 0.6312995521786061
dept_id 0.5721278672947046
store_id 0.6803157356690829
item_id 0.9330058751286124
['state_id', 'cat_id'] 0.6331011075059556
['state_id', 'dept_id'] 0.6571211974699147
['store_id', 'cat_id'] 0.6912383863647799
['store_id', 'dept_id'] 0.7242997080644101
['item_id', 'state_id'] 0.8923241558560997
['item_id', 'store_id'] 0.8506716727404239
0.6969358547652442


In [34]:
valid2_3

,d_1577,d_1578,d_1579,d_1580,d_1581,d_1582,d_1583,d_1584,d_1585,d_1586,d_1587,d_1588,d_1589,d_1590,d_1591,d_1592,d_1593,d_1594,d_1595,d_1596,d_1597,d_1598,d_1599,d_1600,d_1601,d_1602,d_1603,d_1604
0,0.803358,0.615066,0.577915,0.570477,0.560131,0.648474,0.792262,0.803502,0.666149,0.624754,0.583063,0.512698,0.554738,0.603672,0.559934,0.440905,0.408819,0.416149,0.380881,0.429227,0.480009,0.464735,0.353833,0.334217,0.300588,0.266041,0.284645,0.275572
1,0.155629,0.138596,0.138596,0.138596,0.138596,0.145739,0.156007,0.186293,0.140841,0.164548,0.163252,0.163252,0.209213,0.243552,0.241296,0.176922,0.175946,0.197226,0.195672,0.217864,0.238515,0.277213,0.195672,0.302266,0.281941,0.314576,0.364535,0.428647
2,0.210297,0.159598,0.145483,0.140317,0.163982,0.181943,0.230553,0.218976,0.164512,0.189151,0.187660,0.185922,0.209627,0.261030,0.243584,0.185922,0.184897,0.184897,0.209094,0.231592,0.301436,0.300951,0.263008,0.256533,0.255441,0.275474,0.327251,0.386556
3,3.157815,2.105281,2.183979,2.127696,2.116534,2.439387,3.146629,3.365574,2.322358,2.264203,2.277825,2.161263,2.341851,3.490294,3.305017,2.190558,2.083488,2.040980,2.149322,2.445735,3.152508,3.024087,2.138436,2.091360,2.003285,1.986298,2.346117,3.001330
4,1.251815,0.998307,0.947844,1.003341,0.983298,1.127082,1.549153,1.541354,1.240064,1.209521,1.175611,1.071763,1.179024,1.463035,1.396914,0.945253,0.890837,0.842656,0.822047,0.889189,1.014280,0.956246,0.644098,0.600271,0.573956,0.560675,0.620101,0.750664
5,0.907835,0.754453,0.749297,0.701636,0.702196,0.876321,1.150626,1.100297,0.908639,0.779647,0.809978,0.719594,0.803803,1.032023,1.004282,0.758114,0.743638,0.743226,0.744612,0.924635,1.194818,1.143652,0.891413,0.878569,0.886814,1.044269,0.970651,1.203718
6,0.303199,0.230253,0.222274,0.212030,0.201916,0.262224,0.280897,0.266774,0.202741,0.202960,0.202960,0.202457,0.258997,0.332401,0.328972,0.249611,0.278859,0.275202,0.260618,0.364740,0.390155,0.354883,0.314139,0.289278,0.281797,0.281797,0.294843,0.393741
7,7.439828,7.212243,7.575798,6.462459,6.688817,7.726403,7.796329,7.672911,8.384422,7.987253,7.755190,6.575193,7.972352,8.139271,7.024764,8.507703,7.364605,8.846399,8.106058,7.759609,8.215871,8.624268,7.169353,7.204572,7.260961,6.783502,6.398958,7.632296
8,1.135143,0.784706,0.747480,0.748269,0.669496,0.669675,0.763020,0.758467,0.610104,0.567760,0.531979,0.506049,0.533897,0.620028,0.640271,0.469120,0.385068,0.386295,0.344603,0.469045,0.543673,0.606056,0.493953,0.474411,0.470987,0.471747,0.529061,0.596745
9,0.683813,0.523129,0.498184,0.493856,0.507573,0.638775,0.832100,0.806996,0.649489,0.624685,0.659540,0.622615,0.680769,0.828089,0.812454,0.585657,0.553451,0.571428,0.571899,0.611108,0.719435,0.743485,0.469879,0.455574,0.517573,0.506956,0.586245,0.701722


In [36]:
valid2_2

,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0.881232,0.914970,0.845997,0.860195,1.102141,1.231031,1.233993,0.916541,0.835075,0.809209,0.820791,0.989192,1.152654,0.982707,0.766680,0.757699,0.805101,0.824339,0.983599,1.265859,1.222325,0.985399,0.914723,0.966913,0.899078,1.017773,1.362472,1.189088
1,0.158564,0.138373,0.138332,0.158775,0.182766,0.236440,0.286086,0.222840,0.189562,0.187360,0.207470,0.241674,0.370121,0.355349,0.224811,0.221876,0.194572,0.198222,0.226594,0.300730,0.274573,0.185364,0.179268,0.179456,0.206153,0.233903,0.361764,0.383964
2,0.261300,0.246989,0.247008,0.243996,0.290390,0.602315,0.658774,0.461837,0.534012,0.490287,0.473693,0.505707,0.601961,0.628743,0.489763,0.464132,0.471832,0.455698,0.610554,0.577470,0.567337,0.391875,0.366124,0.366347,0.366347,0.438862,0.596526,0.558192
3,1.607565,1.464684,1.527155,1.653394,1.705154,1.981200,2.293302,1.549889,1.338064,1.266908,1.211880,1.350832,1.649356,1.602355,1.284522,1.095091,1.145500,1.261950,1.506480,1.972234,1.797297,1.430874,1.251997,1.314218,1.484431,1.585274,2.233566,2.437993
4,0.960657,0.981672,0.892393,0.900802,1.041961,1.273959,1.194499,0.969535,0.881607,0.914929,0.835316,1.018807,1.359830,1.202589,0.930709,0.849712,0.813339,0.780238,0.976712,1.358098,1.381585,1.047014,1.045044,1.064950,1.045167,1.067129,1.387335,1.272553
5,1.129171,1.255521,1.242437,1.280193,1.544906,1.686525,1.478743,1.262848,1.233719,1.247640,1.230552,1.246123,1.420224,1.269151,1.150042,1.117616,1.157568,1.251129,1.239882,1.485290,1.312480,1.159523,1.222874,0.971698,0.998595,1.221305,1.370857,1.311042
6,0.281000,0.269949,0.268964,0.292414,0.335110,0.362055,0.343836,0.248686,0.236127,0.234932,0.227564,0.308633,0.452397,0.476862,0.357926,0.339581,0.369170,0.353164,0.451708,0.526624,0.519557,0.424228,0.402626,0.384413,0.419615,0.446791,0.522128,0.506917
7,7.710331,7.287882,6.435895,6.652714,6.512217,9.538022,7.303865,6.223241,6.545761,5.875349,6.380367,5.996508,10.141323,7.223671,5.817324,6.236604,5.530021,6.755872,7.111835,9.041278,8.260436,6.204531,5.774264,6.137276,5.025510,6.410245,7.216141,5.667696
8,0.177488,0.172714,0.362106,0.312882,0.326717,0.389574,0.373974,0.346714,0.325339,0.320272,0.305295,0.326906,0.383462,0.365805,0.526940,0.405041,0.347312,0.361554,0.403526,0.503869,0.485348,0.363712,0.335946,0.342963,0.322248,0.383027,0.461372,0.461089
9,0.608173,0.546116,0.560787,0.546291,0.700575,0.810503,0.786397,0.539959,0.542819,0.481914,0.409674,0.466390,0.647145,0.635913,0.512446,0.442283,0.416338,0.376268,0.388875,0.588605,0.702284,0.513951,0.485381,0.441409,0.398390,0.465336,0.522799,0.531112
